In [1]:
import os
import pandas as pd
import time

from google.cloud import automl_v1beta1
from google.cloud.automl_v1beta1.proto import service_pb2

In [2]:
def get_prediction(content, project_id, model_id):
    prediction_client = automl_v1beta1.PredictionServiceClient()

    name = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)
    payload = {'image': {'image_bytes': content }}
    params = {}
    request = prediction_client.predict(name, payload, params)
    return request  # waits till request is returned

In [3]:
project_id = 'YOUR_PROJECT_ID'
model_id = 'YOUR_MODEL_ID'

predictions = []

tic = time.time()
for i, image in enumerate(os.listdir('Data/test/')):
    if i % 100 == 0:
        toc = time.time()
        print("Running image {}/{}. Cumulative run time {:.1f} minutes.".format(
            i, len(os.listdir('Data/test/')), (toc-tic)/60))
    
    file_path = 'Data/test/%s' % image
    
    with open(file_path, 'rb') as ff:
        content = ff.read()

    pred = get_prediction(content, project_id,  model_id)
    try:
        label = pred.payload[0].display_name
        predictions.append(label)
    except:
        print("Failed to identify image {}. Labelling as loose_silky_bent (most populous class).".format(image))
        predictions.append('loose_silky_bent')
    
toc = time.time()
print("Completed! Total run time {:.1f}s.".format(toc-tic))

Running image 0/794. Cumulative run time 0.0 minutes.
Failed to identify image 01291174f.png. Labelling as loose_silky_bent (most populous class).
Failed to identify image 0911d3dee.png. Labelling as loose_silky_bent (most populous class).
Failed to identify image 0c27cf05f.png. Labelling as loose_silky_bent (most populous class).
Failed to identify image 0dba99002.png. Labelling as loose_silky_bent (most populous class).
Failed to identify image 177d7e2a4.png. Labelling as loose_silky_bent (most populous class).
Failed to identify image 1f290e016.png. Labelling as loose_silky_bent (most populous class).
Running image 100/794. Cumulative run time 2.7 minutes.
Failed to identify image 205df1df3.png. Labelling as loose_silky_bent (most populous class).
Failed to identify image 3185294c8.png. Labelling as loose_silky_bent (most populous class).
Failed to identify image 3f826b318.png. Labelling as loose_silky_bent (most populous class).
Running image 200/794. Cumulative run time 5.2 minute

AttributeError: 'float' object has no attribute '1f'

In [4]:
mapping = {
    'black_grass': 'Black-grass',
    'charlock': 'Charlock',
    'cleavers': 'Cleavers',
    'common_chickweed': 'Common Chickweed',
    'common_wheat': 'Common wheat',
    'fat_hen': 'Fat Hen',
    'loose_silky_bent': 'Loose Silky-bent',
    'maize': 'Maize',
    'scentless_mayweed': 'Scentless Mayweed',
    'shepherds_purse': 'Shepherds Purse',
    'small_flowered_cranesbill': 'Small-flowered Cranesbill',
    'sugar_beet': 'Sugar beet'
}

predictions = [mapping[k] for k in predictions]

In [5]:
autoML_preds = pd.DataFrame(columns=['file', 'species'])
autoML_preds['file'] = pd.Series(os.listdir('Data/test/'))
autoML_preds['species'] = pd.Series(predictions)
autoML_preds.head()

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet


In [6]:
autoML_preds.to_csv('autoML.csv', index=False)